In [1]:
import torch  
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


from s9model import Net
from s9utils import create_train_test_sets, show, show_with_gradcam
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")
# model = Net().to(device)
# summary(model, input_size=(3, 32, 32))

## Total params: 5,461,258
## Trainable params: 5,461,258
## Non-trainable params: 0

# Find LR Max, LR Min

In [ ]:
# from s9utils import find_LR_max

# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")

# MEAN = 0.5
# STD = 0.25
# train_dataloader, test_dataloader = create_train_test_sets(MEAN, STD)

# NUM_BATCHES = len(train_dataloader)
# net = Net().to(device)
# criterion = nn.CrossEntropyLoss()

# def lr_lambda(num_steps):
#     return 2*(num_steps+1)
  
# INIT_LR = 0.0001
# optimizer = optim.Adam(net.parameters(), lr=INIT_LR)
# scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# x_lr = []
# y_loss = []

# for k in range(200):
#     x_lr, y_loss = find_LR_max(net, device, train_dataloader, optimizer, criterion, scheduler, x_lr, y_loss)


# plt.figure(figsize=(20,8))
# plt.xscale('log')
# plt.plot(x_lr[0:2450], y_loss[0:2450], color='green', marker='o', linestyle='dashed', linewidth=1, markersize=2)
# plt.show()

## lr max = 0.47, lr min seems like 0.001 is ok i.e. scale factor is 470.

# Main Routine

In [17]:
## initial lr = 0.001
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

MEAN = 0.5
STD = 0.25
train_dataloader, test_dataloader = create_train_test_sets(MEAN, STD)

net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.47, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=24,
                                          pct_start=5/24,
                                          div_factor=470)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
writer = SummaryWriter('/media/App/tsai/logs')

for epoch in range(24):
    print(f'Epoch {epoch}')
    train_acc, train_loss = train(net, device, train_dataloader, optimizer, criterion, scheduler)
    val_acc, val_loss = test(net, device, test_dataloader, criterion)
    writer.add_scalars('resnet', {'train_acc': train_acc, 'val_acc': val_acc}, epoch)
    writer.flush()
    
writer.close()

Epoch 0


/tmp/ipykernel_25245/998129054.py:83: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train loss 1.8756 train accuracy 36.42
Test loss 11.061, test accuracy 20.27

Epoch 1
Train loss 1.3984 train accuracy 50.73
Test loss 1.620, test accuracy 49.97

Epoch 2
Train loss 1.2307 train accuracy 56.814
Test loss 1.595, test accuracy 50.03

Epoch 3
Train loss 1.2688 train accuracy 56.104
Test loss 2.172, test accuracy 43.6

Epoch 4
Train loss 1.2563 train accuracy 56.43
Test loss 1.176, test accuracy 59.98

Epoch 5
Train loss 1.1288 train accuracy 60.746
Test loss 1.044, test accuracy 63.19

Epoch 6
Train loss 1.0220 train accuracy 64.39
Test loss 1.296, test accuracy 58.38

Epoch 7
Train loss 0.9192 train accuracy 68.074
Test loss 1.126, test accuracy 63.72

Epoch 8
Train loss 0.8624 train accuracy 70.37
Test loss 0.726, test accuracy 74.91

Epoch 9
Train loss 0.8196 train accuracy 71.674
Test loss 0.981, test accuracy 67.5

Epoch 10
Train loss 0.7496 train accuracy 74.074
Test loss 1.032, test accuracy 66.15

Epoch 11
Train loss 0.7202 train accuracy 75.072
Test loss 0.877, t

In [ ]:
dir( optim.lr_scheduler.OneCycleLR)